In [ ]:
#this code is not working because of importing package issue

from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_cohere import CohereEmbeddings, ChatCohere
from dotenv import load_dotenv
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever # type: ignore
from langchain.retrievers.document_compressors import LLMChainExtractor # type: ignore

In [ ]:
# Relevant health & wellness documents (plus some unrelated ones for testing filtering)
all_docs = [
    Document(
        page_content="Regular walking boosts heart health and can reduce symptoms of depression.",
        metadata={"source": "H1"}
    ),
    Document(
        page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.",
        metadata={"source": "H2"}
    ),
    Document(
        page_content="Deep sleep is crucial for cellular repair and emotional regulation.",
        metadata={"source": "H3"}
    ),
    Document(
        page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.",
        metadata={"source": "H4"}
    ),
    Document(
        page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.",
        metadata={"source": "H5"}
    ),

    # --- Irrelevant documents (for testing metadata-based filtering) ---
    Document(
        page_content="The solar energy system in modern homes helps balance electricity demand.",
        metadata={"source": "I1"}
    ),
    Document(
        page_content="Python balances readability with power, making it a popular system design language.",
        metadata={"source": "I2"}
    ),
    Document(
        page_content="Photosynthesis enables plants to produce energy by converting sunlight.",
        metadata={"source": "I3"}
    ),
    Document(
        page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.",
        metadata={"source": "I4"}
    ),
    Document(
        page_content="Black holes bend spacetime and store immense gravitational energy.",
        metadata={"source": "I5"}
    ),
]

In [ ]:
embedding_model=CohereEmbeddings(model="embed-english-v3.0")
vector_store=FAISS.from_documents(
    documents=all_docs,
    embedding=embedding_model
)

In [ ]:
base_retriever=vector_store.as_retriever(search_kwargs={"k":5})


In [ ]:
llm=ChatCohere(model="command-a-03-2025")
compressor=LLMChainExtractor.from_llm(llm)
compression_retriever=ContextualCompressionRetriever( 
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [ ]:
query="what is photosynthesis?"
results=compression_retriever.invoke(query)